Хорошая штука для трассировки памяти питонокода, чтоб посмотреть, где утекает память.   
Не редкость, когда протекает [сам питон](https://medium.com/nuances-of-programming/%D0%BF%D0%BE%D0%B8%D1%81%D0%BA-%D0%B8-%D1%83%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5-%D1%83%D1%82%D0%B5%D1%87%D0%B5%D0%BA-%D0%BF%D0%B0%D0%BC%D1%8F%D1%82%D0%B8-%D0%B2-python-570d595fb0d1), но чаще это сторонние пакеты, в т.ч. и мегапопулярные.  
Все на builtins.

In [2]:
import linecache
import os
import tracemalloc

def display_top(snapshot, key_type='lineno', limit=10):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        print("#%s: %s:%s: %.1f KiB"
              % (index, frame.filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

tracemalloc.start()

# целевой код для трассировки
import pandas as pd

snapshot = tracemalloc.take_snapshot()
display_top(snapshot, limit=5)

Top 5 lines
#1: <frozen importlib._bootstrap_external>:647: 12747.5 KiB
#2: /home/user1/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:389: 365.8 KiB
    component.format(**params)
#3: /usr/local/lib/python3.9/re.py:210: 277.5 KiB
    return _compile(pattern, flags).sub(repl, string, count)
#4: /home/user1/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:489: 200.8 KiB
    func.__doc__ = dedent(self.join.join(docitems))
#5: <__array_function__ internals>:3: 183.7 KiB
10584 other: 6641.7 KiB
Total allocated size: 20416.9 KiB


In [3]:
import numpy as np

df = pd.DataFrame({'a': np.random.rand(5), 
              'b': list('hello'), 
              'c': np.linspace(0,1,5)})
df

,a,b,c
0,0.019777,h,0.00
1,0.690853,e,0.25
2,0.022045,l,0.50
3,0.023700,l,0.75
4,0.565270,o,1.00


In [4]:
from sklearn.datasets import load_iris

iris = load_iris()

data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [5]:
snapshot = tracemalloc.take_snapshot()
display_top(snapshot, limit=5)

Top 5 lines
#1: <frozen importlib._bootstrap_external>:647: 25144.3 KiB
#2: /home/user1/envs/py39/lib/python3.9/site-packages/scipy/_lib/doccer.py:66: 675.7 KiB
    return docstring % indented
#3: /usr/local/lib/python3.9/linecache.py:137: 582.2 KiB
    lines = fp.readlines()
#4: /home/user1/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:389: 365.8 KiB
    component.format(**params)
#5: /usr/local/lib/python3.9/re.py:210: 277.5 KiB
    return _compile(pattern, flags).sub(repl, string, count)
23873 other: 16626.7 KiB
Total allocated size: 43672.3 KiB


Объем аллоцированной памяти увеличилися до 40+ Мб